## Important Note
Some manual data manipulation occurred on this dataset! If you're attempting to data clean from start (initial scrape) to finish, there may be some discrepancies.

In [1]:
#Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

carrierDF=pd.read_csv('../data/allCarrierData_merged_years.csv')
print(len(carrierDF))

2024


In [2]:
# Drop duplicates
carrierDF.drop_duplicates(keep='first', inplace=True ) # drop whole duplicate rows
#carrierDF[carrierDF.duplicated(keep=False)]

In [3]:


carrierDF['checker']=carrierDF.Bulletin_Level.apply(lambda x: str(x)[:10])
carrierDF['checker']=carrierDF.checker.apply(lambda x: str(x)[-2:])
print(carrierDF[carrierDF.checker!="01"])

      Carrier Manufacture                Model Release_Date  \
1        AT&T     Alcatel   Alcatel IdealXCITE   2018-04-03   
4        AT&T     Alcatel   Alcatel IdealXCITE   2018-12-18   
5        AT&T  BlackBerry    BlackBerry KEYone   2017-09-26   
7        AT&T  BlackBerry    BlackBerry KEYone   2017-10-31   
8        AT&T  BlackBerry    BlackBerry KEYone   2017-11-15   
...       ...         ...                  ...          ...   
1949  Verizon     Samsung  Galaxy Tab 4 (10.1)   2017-01-12   
1951  Verizon     Samsung   Galaxy Tab 4 (8.0)   2017-01-12   
1974  Verizon     Samsung         Galaxy Tab S   2017-01-19   
1996  Verizon        Sony           Xperia Z3v   2016-03-14   
1998  Verizon     Verizon          Ellipsis 10   2017-07-14   

           Bulletin_Level  difference Android_Level_Guessed  \
1              2018-02-05        57.0                 False   
4              2018-10-05        74.0                 False   
5              2017-09-05        21.0                 

In [4]:
######## Filter down to approprate timeline ########
print(len(carrierDF))

carrierDF.Bulletin_Level=carrierDF.Bulletin_Level.astype(str)
carrierDF.Bulletin_Level=carrierDF.Bulletin_Level.str.replace(" [0:]*","")
carrierDF.Bulletin_Level=carrierDF.Bulletin_Level.apply(lambda x: str(x[:-2])+"01") # convert all bulletins to first day

# setting the bad bulletins
#badBulletins=['2019-09-01', '2019-08-01', '2019-07-01']

# for badBul in badBulletins: # filtering...
#     carrierDF=carrierDF[carrierDF.Bulletin_Level!=badBul]
print(len(carrierDF))

1975
1975


In [5]:
carrierDF[carrierDF.duplicated(subset=['Carrier','Manufacture','Model','Bulletin_Level'],keep=False)]

,Carrier,Manufacture,Model,Release_Date,Bulletin_Level,difference,Android_Level_Guessed,build,Year_Phone_Release,Month_Phone_Release,checker
5,AT&T,BlackBerry,BlackBerry KEYone,2017-09-26,2017-09-01,21.0,False,MPSS.TA.2.3.c1-00333-8953_GEN_PACK-1.103777.4....,2017,4,05
6,AT&T,BlackBerry,BlackBerry KEYone,2017-09-26,2017-09-01,25.0,False,MPSS.TA.2.3.c1-00333-8953_GEN_PACK-1.103777.4....,2017,4,01
56,AT&T,LG,G5,2017-12-05,2017-11-01,29.0,False,MPSS.TH.2.0.c1.4-00041-M8996FAAAANAZM-1.109835...,2016,4,06
57,AT&T,LG,G5,2017-12-12,2017-11-01,41.0,False,MPSS.TH.2.0.c1.4-00013-M8996FAAAANAZM-2.44625....,2016,4,01
68,AT&T,LG,G8 ThinQ,2019-11-08,2019-11-01,7.0,True,nan nan G820UM10m,2019,4,01
...,...,...,...,...,...,...,...,...,...,...,...
1951,Verizon,Samsung,Galaxy Tab 4 (8.0),2017-01-12,2017-01-01,7.0,True,NaN,2014,6,05
1973,Verizon,Samsung,Galaxy Tab S,2017-01-19,2017-01-01,18.0,True,NaN,2014,6,01
1974,Verizon,Samsung,Galaxy Tab S,2017-01-19,2017-01-01,14.0,True,NaN,2014,6,05
1997,Verizon,Verizon,Ellipsis 10,2017-07-14,2017-07-01,13.0,True,NaN,2015,11,01


In [6]:
# Remove duplicate carrier, manufacture, phone, bulletin pairings. Keep the earliest release date
carrierDF=carrierDF.sort_values(by=['Carrier','Manufacture','Model','Bulletin_Level','difference'])# Sort to properly drop duplicates if bulletins were normalized to the same (e.g., level 1 and 5 were both patched thus keep the first date)
carrierDF.drop_duplicates(subset=['Carrier','Manufacture','Model','Bulletin_Level'],keep='first', inplace=True ) # drop the duplicates and keep the lowest difference
print(len(carrierDF))

for car in carrierDF.Carrier.unique():
    print(car)
    print(len(carrierDF[carrierDF.Carrier==car]))

1939
AT&T
394
Sprint
364
TMobile
658
Verizon
523


In [7]:
# errors in parsing verizon...
carrierDF=carrierDF.sort_values(by=['Carrier','Manufacture','Model','Release_Date','Bulletin_Level'])
carrierDF.drop_duplicates(subset=['Carrier','Manufacture','Model','Release_Date','Bulletin_Level'],keep='first', inplace=True ) 

print(len(carrierDF))

1939


In [8]:
# Recalculate difference!
carrierDF.Bulletin_Level=pd.to_datetime(carrierDF.Bulletin_Level)
carrierDF.Release_Date=pd.to_datetime(carrierDF.Release_Date)

newDif=carrierDF.Release_Date-carrierDF.Bulletin_Level
newDif=newDif.apply(lambda x: x.days)
carrierDF.difference=newDif.copy()
print(carrierDF.head())

  Carrier Manufacture               Model Release_Date Bulletin_Level  \
0    AT&T        ASUS      ASUS PadFone X   2018-08-31     2018-04-01   
1    AT&T     Alcatel  Alcatel IdealXCITE   2018-04-03     2018-02-01   
2    AT&T     Alcatel  Alcatel IdealXCITE   2018-04-03     2018-04-01   
3    AT&T     Alcatel  Alcatel IdealXCITE   2018-10-24     2018-08-01   
4    AT&T     Alcatel  Alcatel IdealXCITE   2018-12-18     2018-10-01   

   difference Android_Level_Guessed  \
0         152                  True   
1          61                 False   
2           2                  True   
3          84                 False   
4          78                 False   

                                         build  Year_Phone_Release  \
0                 nan ATT_11.10.11.2File 3.4.0                2014   
1  JO.2.0.c1-00150-8909_GEN_PACK-1 nan YAMUAM0                2017   
2                          nan YB4UA40File nan                2017   
3  JO.2.0.c1-00267-8909_GEN_PACK-2 nan YAYUAY0

In [9]:
# Remove outliers!!
index=carrierDF[((carrierDF.Model=='Galaxy A6') & (carrierDF.difference==-312))].index
carrierDF.at[index,'Release_Date']=pd.to_datetime("2019-05-24")

index=carrierDF[((carrierDF.Model=='Galaxy Tab A 8.0 (2018)') & (carrierDF.difference==-71))].index
carrierDF.at[index,'Release_Date']=pd.to_datetime("2019-06-22")

#The following is a use case from using the assumption of the "same" year eventhough Jan starts a new year
index=carrierDF[((carrierDF.Model=='Galaxy Note9') & (carrierDF.difference==-329))].index
carrierDF.at[index,'Bulletin_Level']=pd.to_datetime("2018-12-01")


# Recalculate readjusted outliers
carrierDF.Bulletin_Level=pd.to_datetime(carrierDF.Bulletin_Level)
carrierDF.Release_Date=pd.to_datetime(carrierDF.Release_Date)

newDif=carrierDF.Release_Date-carrierDF.Bulletin_Level
newDif=newDif.apply(lambda x: x.days)
carrierDF.difference=newDif.copy()

carrierDF=carrierDF[carrierDF.difference>=-10]
carrierDF=carrierDF[carrierDF.difference<=150]


# Now change T-Mobile manufacturer to Revvl (T-Mobile)
carrierDF.Manufacture=carrierDF.Manufacture.str.replace("T-Mobile","Revvl(T-Mobile)")
carrierDF.Carrier=carrierDF.Carrier.str.replace("TMobile","T-Mobile")


In [10]:
print(len(carrierDF))
print(len(carrierDF.Model.unique()))
print(len(carrierDF.Manufacture.unique()))
print()
for car in carrierDF.Carrier.unique():
    print(car)
    print(len(carrierDF[carrierDF.Carrier==car]))

1920
268
26

AT&T
392
Sprint
363
T-Mobile
657
Verizon
508


In [11]:
carrierDF.sort_values(by=['Carrier','Manufacture','Model','Release_Date'],inplace=True)
carrierDF.to_csv('../data/allCarrierData_final.csv',index=False)

In [12]:
print(carrierDF.head())

  Carrier Manufacture               Model Release_Date Bulletin_Level  \
1    AT&T     Alcatel  Alcatel IdealXCITE   2018-04-03     2018-02-01   
2    AT&T     Alcatel  Alcatel IdealXCITE   2018-04-03     2018-04-01   
3    AT&T     Alcatel  Alcatel IdealXCITE   2018-10-24     2018-08-01   
4    AT&T     Alcatel  Alcatel IdealXCITE   2018-12-18     2018-10-01   
5    AT&T  BlackBerry   BlackBerry KEYone   2017-09-26     2017-09-01   

   difference Android_Level_Guessed  \
1          61                 False   
2           2                  True   
3          84                 False   
4          78                 False   
5          25                 False   

                                               build  Year_Phone_Release  \
1        JO.2.0.c1-00150-8909_GEN_PACK-1 nan YAMUAM0                2017   
2                                nan YB4UA40File nan                2017   
3        JO.2.0.c1-00267-8909_GEN_PACK-2 nan YAYUAY0                2017   
4        JO.2.0.c1-002

In [13]:
print(carrierDF.Manufacture.unique())

['Alcatel' 'BlackBerry' 'HTC' 'Huawei' 'Kyocera' 'LG' 'Motorola' 'RED'
 'Razer' 'Samsung' 'ZTE' 'CAT' 'Coolpad' 'Essential' 'Google' 'Slate'
 'Sonim' 'ASUS' 'OnePlus' 'Revvl(T-Mobile)' 'Sony' 'KYOCERA' 'Nokia'
 'Orbic' 'Palm' 'Verizon']


In [14]:
# print(carrierDF[carrierDF.Model.str.contains("Gizmo")])